In [1]:
# !pip3 install huggingface
# !pip3 install huggingface_hub
# !pip3 install datasets
# !pip3 install kaggle
# !pip3 install pandas
# !pip3 install matplotlib
# !pip3 install transformers

# Kaggle 

In [ ]:
IN_COLAB = 'google.colab' in str(get_ipython())
print(f"in colab - {IN_COLAB}")

if IN_COLAB:
    !rm /root/.kaggle
    !mkdir -p /root/.kaggle
    !cp ./kaggle.json /root/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json

In [2]:
from zipfile import ZipFile
import os
import shutil
from zipfile import ZipFile

from kaggle.api.kaggle_api_extended import KaggleApi

DATA_DIR = './_data'
JIGSAW_DIR = DATA_DIR+'/jigsaw'


os.makedirs(JIGSAW_DIR, exist_ok = True)

zip_file = DATA_DIR+'/jigsaw-toxic-comment-classification-challenge.zip'
if not os.path.exists(zip_file):
    k_api = KaggleApi()
    k_api.authenticate()
    
    k_api.dataset_download_files(
        dataset='julian3833/jigsaw-toxic-comment-classification-challenge',
        unzip=False,
        path=DATA_DIR
    )

zf = ZipFile(zip_file)
zf.extractall(JIGSAW_DIR) 
zf.close()

JIGSAW_TRAIN_PATH = JIGSAW_DIR+'/train.csv'
JIGSAW_TEST_X_PATH = JIGSAW_DIR+'/test.csv'
JIGSAW_TEST_Y_PATH = JIGSAW_DIR+'/test_labels.csv'


In [3]:
import pandas as pd

df  = pd.read_csv(JIGSAW_TRAIN_PATH)

In [4]:
from typing import Any
from transformers import RobertaTokenizer
from transformers import RobertaModel

from torch import nn

class Encoder(nn.Module):
    def __init__(self) -> None:
        super(Encoder, self).__init__()
        
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.model = RobertaModel.from_pretrained('roberta-base')
        self.model.eval()
    
    def forward(self, texts):
        tokens = self.tokenizer(
            texts, 
            return_tensors='pt', 
            padding=True, 
            truncation=True)
        embeddings = self.model(**tokens)['pooler_output']
        return embeddings
        
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(args[0])

/Users/n0c09jf/anaconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch.nn as nn

class MultiLabelClassificationHead(nn.Module):
    def __init__(self, hidden_size, nlabels) -> None:
        super(MultiLabelClassificationHead, self).__init__()
        self.hidden_size = hidden_size
        self.nlabels = nlabels
        
        self.input = nn.Linear(
            in_features=self.hidden_size, 
            out_features=self.hidden_size, bias=True)
        
        def label_model():
            return nn.Sequential(
                nn.Dropout(p=0.1, inplace=False),
                nn.Linear(in_features=self.hidden_size,
                          out_features=2, bias=True)
            )
            
        self.heads = nn.ModuleList([
            label_model() for label in range(0, nlabels)
        ])
    
    def forward(self, x):
        x = self.input(x)
        
        y_pred = []
        for m in self.heads:
            y_pred.append(m(x))
        
        return y_pred

In [6]:
from torch.utils.data import Dataset
import torch
import numpy as np

class ToxicDataset(Dataset):
    def __init__(self, dataframe, encoder, labels) -> None:
        super(ToxicDataset, self).__init__()
        self.dataframe = dataframe
        self.encoder = encoder
        self.labels = labels
    
    def __getitem__(self, index) -> Any:
        text = self.dataframe.iloc[index]['comment_text']
        
        embedding = self.encoder([text])[0]
        labels = self.dataframe.iloc[index][self.labels].to_numpy()
        
        return embedding, torch.from_numpy(labels.astype(np.float32))
    
    def __len__(self):
        return len(self.dataframe)

In [7]:
from torch.utils.data import DataLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

encoder_model = Encoder()
classify_model = MultiLabelClassificationHead(768, len(labels)).to(device)

BATCH_SIZE = 64
def to_dataloader(dataframe):
    ds = ToxicDataset(dataframe, encoder_model, labels)
    dl = DataLoader(
        ds, batch_size=BATCH_SIZE, shuffle=True)
    return dl

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import os

MODELS_DIR = './_models'
JIGSAW_MODEL_DIR = './_models/jigsaw'
os.makedirs(JIGSAW_MODEL_DIR, exist_ok=True)


def save_model(model, name) -> None:
    torch.save(
        model.state_dict(), JIGSAW_MODEL_DIR+'/'+name)


def load_model(model, name) -> Any:
    if os.path.exists(JIGSAW_MODEL_DIR+'/'+name):
        model.load_state_dict(torch.load(JIGSAW_MODEL_DIR+'/'+name))
    else:
        print(f"{name} not found!")
    return model

In [9]:
import torch
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report, hamming_loss
from sklearn.metrics import average_precision_score

def train_epoch(model, dataloader, lossFn, optimizer):
    epoch_loss = 0
    model.train()
    for input_tensor, target_tensor in dataloader:
        optimizer.zero_grad()
        
        input_tensor = input_tensor.to(device)
        target_tensor = target_tensor.to(device)
        
        probs = model(input_tensor)
        loss = lossFn(probs, target_tensor)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.detach().item()

    epoch_mean_loss = epoch_loss/len(dataloader)
    return epoch_mean_loss

def val_epoch(model, dataloader, lossFn):
    epoch_loss = 0
    model.eval()

    y_pred = []
    y_actual = []
    
    with torch.no_grad():
        for input_tensor, target_tensor in dataloader:
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)
        
            logits = model(input_tensor)
            
            loss = lossFn(logits, target_tensor)
            epoch_loss += loss.detach().item()
            
            predictions = []
            for logit in logits:
                _, predicted = torch.max(logit,1)
                predictions.append(torch.Tensor.cpu(predicted).detach())
            
            y_pred.extend(list(np.column_stack(predictions)))
            y_actual.extend(list(torch.Tensor.cpu(target_tensor.detach()).numpy()))

    epoch_mean_loss = epoch_loss/len(dataloader)
    return epoch_mean_loss, y_pred, y_actual

def compute_metrics(y_actual, y_pred):
    hl = hamming_loss(y_actual, y_pred)
    map = average_precision_score(y_actual, y_pred, average='micro')
    wap = average_precision_score(y_actual, y_pred, average='weighted')
    return hl, map, wap

In [10]:
from torch.nn import CrossEntropyLoss
import numpy as np

cel = CrossEntropyLoss()

def loss_fn(predictions, actuals):
    losses = 0
    for i, prediction in enumerate(predictions):
        losses += cel(prediction, actuals[:,i].long())
    return losses
    
optim = torch.optim.Adam(classify_model.parameters(), lr=0.001)

sample_df = df[0:2]
dl = to_dataloader(sample_df)
loss = train_epoch(classify_model, dl, loss_fn, optim)
print(loss)

_, y_pred, y_actual = val_epoch(classify_model, dl, loss_fn)
print(y_pred)
print(y_actual)
print(compute_metrics(y_actual, y_pred))

4.185133457183838
[array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0])]
[array([0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0.], dtype=float32)]
(0.0, 0.0, 0)


/Users/n0c09jf/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [11]:
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

train_df, val_df = train_test_split(df[0:1000], test_size=0.1)
train_dl = to_dataloader(train_df)
val_dl = to_dataloader(val_df)

classify_model = load_model(classify_model, '1000-best-MultiLabelClassificationHead')
classify_model.to(device)
optim = torch.optim.Adam(classify_model.parameters(), lr=0.1)
scheduler = ReduceLROnPlateau(optim, 'max')

epochs = 100

train_losses = []
val_losses = []
for e in range(1, epochs+1):
    train_loss = train_epoch(classify_model, train_dl, loss_fn, optim)
    val_loss, y_pred, y_actual = val_epoch(classify_model, val_dl, loss_fn)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    measure = compute_metrics(y_actual, y_pred)
    scheduler.step(measure[2])
    
    if e % 5 == 0:
        print("epoch-{}: losses - train {}, val {}; (hl,map,wap): ({}, {}, {}))".format( 
            e, round(train_loss, 4), 
            round(val_loss, 4), 
            round(measure[0], 4), 
            round(measure[1], 4), 
            round(measure[2], 4)
        ))
        save_model(classify_model, str(e)+'-'+classify_model._get_name())

epoch-5: losses - train 505.3515, val 553.1869; (hl,map,wap): (0.315, 0.0917, 0.0961))
epoch-10: losses - train 17.5505, val 27.5977; (hl,map,wap): (0.045, 0.2548, 0.341))
epoch-15: losses - train 16.1883, val 338.8826; (hl,map,wap): (0.685, 0.0743, 0.1004))
epoch-20: losses - train 132.3013, val 95.1565; (hl,map,wap): (0.1083, 0.2006, 0.3228))
epoch-25: losses - train 315.0006, val 152.1738; (hl,map,wap): (0.0533, 0.3459, 0.3612))
epoch-30: losses - train 25.2111, val 65.6796; (hl,map,wap): (0.0333, 0.4366, 0.5003))
epoch-35: losses - train 24.4281, val 56.2684; (hl,map,wap): (0.03, 0.499, 0.5365))
epoch-40: losses - train 27.997, val 40.8532; (hl,map,wap): (0.0283, 0.5159, 0.554))
epoch-45: losses - train 21.3084, val 49.4977; (hl,map,wap): (0.03, 0.499, 0.5566))
epoch-50: losses - train 21.0378, val 39.6309; (hl,map,wap): (0.03, 0.499, 0.5566))
epoch-55: losses - train 18.8468, val 35.8177; (hl,map,wap): (0.03, 0.499, 0.5566))


KeyboardInterrupt: 